<a href="https://colab.research.google.com/github/hamzafarooq/building-llm-applications-from-scratch/blob/main/Module%201/Module_1_Foundation_LLM_Maven_v2_module_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!uv add nltk

Resolved 78 packages in 12ms
Audited 58 packages in 7ms


In [6]:
# code courtesy of https://nlpforhackers.io/language-models/
# Import necessary libraries
import nltk
from nltk.corpus import reuters
from nltk import bigrams, trigrams
from collections import Counter, defaultdict

# Download required NLTK data
nltk.download("reuters")
nltk.download("punkt")
nltk.download("punkt_tab")

# Create a placeholder for the language model
# Using nested defaultdict to automatically handle new keys
model = defaultdict(lambda: defaultdict(lambda: 0))

# Build the trigram model
for sentence in reuters.sents():
    # Generate trigrams from each sentence
    # pad_right and pad_left add None at the beginning and end of the sentence
    for w1, w2, w3 in trigrams(sentence, pad_right=True, pad_left=True):
        # Increment the count for this trigram
        model[(w1, w2)][w3] += 1

# Convert frequency counts to probabilities
for w1_w2 in model:
    # Calculate total count for this bigram
    total_count = float(sum(model[w1_w2].values()))

    # Convert each count to a probability
    for w3 in model[w1_w2]:
        model[w1_w2][w3] /= total_count

# At this point, model contains the probabilities of each word (w3)
# given the previous two words (w1, w2)

[nltk_data] Downloading package reuters to
[nltk_data]     /Users/yangrchen/nltk_data...
[nltk_data]   Package reuters is already up-to-date!
[nltk_data] Downloading package punkt to /Users/yangrchen/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/yangrchen/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


This code creates a trigram language model using the Reuters corpus. Here's a breakdown of what it does:

*   It imports necessary libraries and downloads required NLTK data.
*   It creates a nested defaultdict to store the model. This allows for easy counting of trigrams without explicitly checking if a key exists.
*   It iterates through all sentences in the Reuters corpus, generating trigrams for each sentence.
*   For each trigram, it increments the count in the model.
*   After counting all trigrams, it converts the counts to probabilities by dividing each count by the total count for its corresponding bigram.


The resulting model can be used to predict the probability of a word given the two preceding words.

In [7]:
# Get the probabilities for words following "the price" and sort them
# This creates a list of (word, probability) tuples, sorted by probability in descending order
sorted_probabilities = sorted(
    dict(model["the", "price"]).items(), key=lambda x: x[1], reverse=True
)

# Print the sorted probabilities
print("Most probable words following 'the price', in order:")
for word, prob in sorted_probabilities:
    print(f"{word}: {prob}")

Most probable words following 'the price', in order:
of: 0.3209302325581395
it: 0.05581395348837209
to: 0.05581395348837209
for: 0.05116279069767442
.: 0.023255813953488372
at: 0.023255813953488372
adjustment: 0.023255813953488372
is: 0.018604651162790697
,: 0.018604651162790697
paid: 0.013953488372093023
increases: 0.013953488372093023
per: 0.013953488372093023
the: 0.013953488372093023
will: 0.013953488372093023
cut: 0.009302325581395349
cuts: 0.009302325581395349
(: 0.009302325581395349
differentials: 0.009302325581395349
has: 0.009302325581395349
stayed: 0.009302325581395349
was: 0.009302325581395349
freeze: 0.009302325581395349
increase: 0.009302325581395349
would: 0.009302325581395349
yesterday: 0.004651162790697674
effect: 0.004651162790697674
used: 0.004651162790697674
climate: 0.004651162790697674
reductions: 0.004651162790697674
limit: 0.004651162790697674
now: 0.004651162790697674
moved: 0.004651162790697674
adjustments: 0.004651162790697674
slumped: 0.004651162790697674
mov

In [8]:
!uv add pytorch-transformers --quiet

In [9]:
# Import required libraries
import torch
from pytorch_transformers import GPT2Tokenizer, GPT2LMHeadModel

# Load pre-trained model tokenizer (vocabulary)
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")


/Users/yangrchen/dev/self_learning/building-llm-applications-from-scratch/.venv/lib/python3.13/site-packages/pytorch_transformers/tokenization_xlm.py:679: SyntaxWarning: invalid escape sequence '\*'
  \* The original XLM used [Stanford Segmenter](https://nlp.stanford.edu/software/stanford-segmenter-2018-10-16.zip).
100%|██████████| 456318/456318 [00:00<00:00, 517162.70B/s]


In [10]:
# Define the input text
text = "I am thinking"
print(f"Input text: {text}")

# Encode the input text
indexed_tokens = tokenizer.encode(text)

# Convert indexed tokens to a PyTorch tensor
tokens_tensor = torch.tensor([indexed_tokens])

# Load pre-trained model (weights)
model = GPT2LMHeadModel.from_pretrained("gpt2")

# Set the model in evaluation mode to deactivate the DropOut modules
model.eval()

# Check if CUDA is available and move model and tensors to GPU if possible
if torch.cuda.is_available():
    device = torch.device("cuda")
elif torch.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")

tokens_tensor = tokens_tensor.to(device)
model.to(device)

print(f"Using device: {device}")


Input text: I am thinking


100%|██████████| 548118077/548118077 [01:26<00:00, 6309052.78B/s] 


Using device: mps


In [11]:
# Predict next token
with torch.no_grad():
    outputs = model(tokens_tensor)
    predictions = outputs[0]

# Get the predicted next sub-word (token)
predicted_index = torch.argmax(predictions[0, -1, :]).item()
predicted_token = tokenizer.decode([predicted_index])

# Add the predicted token to the original text
predicted_text = text + predicted_token

# Print the results
print(f"Predicted next token: '{predicted_token}'")
print(f"Complete predicted text: '{predicted_text}'")

Predicted next token: ' of'
Complete predicted text: 'I am thinking of'


In [23]:
tokenizer.decode(torch.argmax(predictions[0, -1, :]).item())

' of'

#Assignment -- Extend the code above to a sentence

In [26]:
n_tokens = 20
generated_text = text
for _ in range(n_tokens):
    indexed_tokens = tokenizer.encode(generated_text)
    tokens_tensor = torch.tensor([indexed_tokens]).to(device)
    
    with torch.no_grad():
        outputs = model(tokens_tensor)
        predictions = outputs[0]
    
    predicted_index = torch.argmax(predictions[0, -1, :]).item()
    predicted_token = tokenizer.decode([predicted_index])

    generated_text += predicted_token

    print(f"Predicted next token: '{predicted_token}'")
    print(f"Full generated text: '{generated_text}'")


Predicted next token: ' of'
Full generated text: 'I am thinking of'
Predicted next token: ' doing'
Full generated text: 'I am thinking of doing'
Predicted next token: ' a'
Full generated text: 'I am thinking of doing a'
Predicted next token: ' book'
Full generated text: 'I am thinking of doing a book'
Predicted next token: ' about'
Full generated text: 'I am thinking of doing a book about'
Predicted next token: ' the'
Full generated text: 'I am thinking of doing a book about the'
Predicted next token: ' history'
Full generated text: 'I am thinking of doing a book about the history'
Predicted next token: ' of'
Full generated text: 'I am thinking of doing a book about the history of'
Predicted next token: ' the'
Full generated text: 'I am thinking of doing a book about the history of the'
Predicted next token: ' United'
Full generated text: 'I am thinking of doing a book about the history of the United'
Predicted next token: ' States'
Full generated text: 'I am thinking of doing a book a

# Solution


In [ ]:
# Optional: Generate multiple next tokens
num_tokens_to_generate = 10
generated_text = text

for _ in range(num_tokens_to_generate):
    # Encode all text generated so far
    indexed_tokens = tokenizer.encode(generated_text)
    tokens_tensor = torch.tensor([indexed_tokens]).to(device)

    # Predict next token
    with torch.no_grad():
        outputs = model(tokens_tensor)
        predictions = outputs[0]

    # Get the predicted next token
    predicted_index = torch.argmax(predictions[0, -1, :]).item()
    predicted_token = tokenizer.decode([predicted_index])

    # Add the predicted token to the generated text
    generated_text += predicted_token

print(f"\nGenerated text with {num_tokens_to_generate} additional tokens:")
print(generated_text)